In [1]:
from juliacall import Main as jl
from juliacall import Pkg as jlPkg
from juliacall import convert as jlconvert
from matplotlib import pyplot as plt
from random import randint
import networkx as nx
import numpy as np

jlPkg.add("Laplacians")
jl.seval("using Laplacians")
jl.seval("using SparseArrays")

np.set_printoptions(precision=3, suppress=True, formatter={"float": "{: 0.3f}".format})

   Resolving package versions...
  No Changes to `~/Documents/LN-pIRLS/.venv/julia_env/Project.toml`
  No Changes to `~/Documents/LN-pIRLS/.venv/julia_env/Manifest.toml`


In [2]:
def draw_graph(G: nx.Graph, SRC: int, DST: int):
    # Draw graph
    D = dict(G.degree)
    NODELIST = D.keys()
    NODESIZE = [v * 10 for v in D.values()]
    POS = nx.spring_layout(G)
    LABELS = {SRC: SRC, DST: DST}
    nx.draw(G, nodelist=NODELIST, node_size=NODESIZE, pos=POS, width=0.2, alpha=0.5)
    nx.draw_networkx_labels(G, pos=POS, labels=LABELS, font_weight="bold")

    # Highlight shortest path from source to destination
    path = nx.shortest_path(G, SRC, DST)
    sizes = [D[node] * 10 for node in path]
    edges = list(zip(path, path[1:]))
    nx.draw_networkx_nodes(G, pos=POS, nodelist=path, node_size=sizes, node_color="r")
    nx.draw_networkx_edges(G, pos=POS, edgelist=edges, edge_color="r")

    plt.switch_backend("TkAgg")
    plt.show()

In [3]:
# NOTE: For the symmetrical case, which balance do we use?
#       "A" matrix will be very different depending on local
#       or remote balance, and we don't know which one the
#       solver will use ahead of time.

# Generate directed dual-BA graph with 1024 nodes.
SRC: int
DST: int
AMT: int = 1e6
count = 0
while True:
    G = nx.dual_barabasi_albert_graph(1024, 2, 3, 0.5)
    p = nx.periphery(G)
    p = [node for node in p if G.degree[node] >= 3]

    # Pick the furthest pair of nodes
    dist = 0
    for i, s in enumerate(p):
        for d in p[i + 1 :]:
            temp = nx.shortest_path_length(G, s, d)
            if temp > dist:
                SRC, DST, dist = s, d, temp
    if dist >= 2:
        break
    if (count := count + 1) > 100:
        raise Exception("dist too large")


# Assign edge attributes
#  - channel capacities from 1ksat to 10BTC
#  - balances from 0 to channel capacity
#  - proportional fee rates from 0 to 1%
#  - base fees from 0 to 0.01BTC
# NOTE: Consider making these dependent on node degree
for i, j in G.edges():
    G[i][j]["c"] = randint(1e3, 1e9)
    G[i][j]["u"] = randint(0, G[i][j]["c"])
    G[i][j]["r"] = randint(0, 0.01 * 1e6)
    G[i][j]["b"] = randint(0, 0.01 * 1e8)

print(f"Generated graph with {len(G.edges())} edges")
print(f"Source: {SRC}\nDestination: {DST}")

Generated graph with 2561 edges
Source: 118
Destination: 1023


In [6]:
def uncertainty_coefficient(G: nx.Graph, i: int, j: int) -> float:
    return -((2 / G[i][j]["c"]) ** 2) * np.log(1 / 2)


def fee_rate_decimal(G: nx.Graph, i: int, j: int) -> float:
    return G[i][j]["r"] / 1e6


def base_fee_satoshi(G: nx.Graph, i: int, j: int) -> float:
    return G[i][j]["b"] / 1e3


def smoothed_w2(w2: float, fe: float, epsilon: float) -> float:
    return (w2 / 2) * (epsilon + 1 / epsilon) if abs(fe) < epsilon else w2 * abs(fe)


def J(G: nx.Graph, f: np.ndarray) -> float:
    j1 = sum(uncertainty_coefficient(G, i, j) * f[i] ** 2 for i, j in G.edges())
    j2 = sum(fee_rate_decimal(G, i, j) * f[i] for i, j in G.edges())
    return j1 + j2


def np_matrix_to_jl(A: np.ndarray) -> jl.SparseMatrixCSC:
    A = A.tocoo()
    i_jl = jlconvert(T=jl.Vector[jl.Int64], x=A.row + 1)
    j_jl = jlconvert(T=jl.Vector[jl.Int64], x=A.col + 1)
    v_jl = jlconvert(T=jl.Vector[jl.Float64], x=A.data)
    return jl.SparseArrays.sparse(i_jl, j_jl, v_jl, A.shape[0], A.shape[1])


def print_iteration(
    i: int, w1: np.ndarray, w2: np.ndarray, C: np.ndarray, f: np.ndarray
) -> None:
    print(f"==== Iteration {i + 1} ====")
    print(f"\tmin\t\tmax\t\tavg")
    print(f"w1:\t{np.min(w1):+0.3e}\t{np.max(w1):+0.3e}\t{np.mean(w1):+0.3e}")
    print(f"w2:\t{np.min(w2):+0.3e}\t{np.max(w2):+0.3e}\t{np.mean(w2):+0.3e}")

    print(f"C@f[SRC]: {(C @ f)[SRC]:+0.3e}")
    print(f"C@f[DST]: {(C @ f)[DST]:+0.3e}")
    print(f"sum(C@f): {np.sum(C @ f):+0.3e}")
    print(f"J(f): {J(G, f):+0.3e}\n")

In [17]:
# Initial solution with unity weighting
A = np_matrix_to_jl(nx.adjacency_matrix(G))
n = G.number_of_nodes()
m = G.number_of_edges()
d = np.zeros(n)
d[SRC] += AMT
d[DST] -= AMT

solver = jl.Laplacians.approxchol_lap(A, verbose=True, tol=1e-12)
x = solver(d)  # type juliacall.VectorValue
print(x)

C = nx.incidence_matrix(G, oriented=True)
f = C.T @ x

print_iteration(-1, np.ones(m), np.ones(m), C, f)

residual_laplacians = jl.Laplacians.lap(A) * x - d
print(f"Residual norm: {np.linalg.norm(residual_laplacians):0.3e}")

print(A[0:4, 0:4])

[29.23838397172974, 2127.127215848528, -54.02381467543579, -1462.6626086424076, -182.69457330306258, 2734.664148619766, 1934.1231020293205, 787.4427723624663, 2204.902013922431, 607.1513735977154, -1752.8165465957475, 490.46689412737544, 1463.1877996273859, 129.05470007290245, 565.0256615888244, 133.4297999425297, 51329.9933670864, 849.7066622254436, 31.682373399338605, -141.81823296085858, 3940.4850650719195, -292.6442739109399, -0.30243774470604223, -728.3247532883369, 541.9519945138819, -189.4139236642665, 98.17608094157009, -283.4016425730121, 874.7468806332291, -1094.9466939445347, 914.4908334899213, 1238.5719187792852, -348.3211899444992, -2311.0136412870984, 162.74422854930265, 546.962022326832, 835.4273203122411, -166.99518231879, -102.7669530178264, 238.32862609954344, -602.5373711193342, 2989.231675171056, 528.1152745162032, 1816.5735027066714, 589.9160436420134, -4474.577767367171, 1867.9340491072821, 756.543059235975, 777.7264866937052, -2589.629151684552, 868.4029358438036

In [18]:
# Iterate
for iteration in range(1):
    # Assign edge weights
    w1 = np.array([uncertainty_coefficient(G, *tuple(edge)) for edge in G.edges()]).T
    w2 = np.array([base_fee_satoshi(G, *tuple(edge)) for edge in G.edges()]).T
    for e, (i, j) in enumerate(G.edges()):
        G[i][j]["w"] = float(w1[e] ** 2 + smoothed_w2(w2[e], f[e], G[i][j]["c"]))
        # G[i][j]["w"] = 1

    # Get weighted adjacency matrix
    A = np_matrix_to_jl(nx.adjacency_matrix(G, weight="w"))

    # Iterative solution with weighted A matrix
    solver = jl.Laplacians.approxchol_lap(A, verbose=True, tol=1e-12)
    x = solver(d)  # type juliacall.VectorValue
    f = C.T @ x

    print_iteration(iteration, w1, w2, C, f)

    residual_laplacians = jl.Laplacians.lap(A) * x - d
    print(f"Residual norm: {np.linalg.norm(residual_laplacians):0.3e}\n")

    print(A[0:4, 0:4])

==== Iteration 1 ====Using greedy degree ordering. Factorization time: 0.00180816650390625
Factorization size: 10480
Ratio of operator edges to original edges: 2.046075751659508
	min		max		avg
w1:	+2.777e-18	+2.087e-11	+1.111e-14
w2:	+3.320e-01	+1.000e+03	+5.025e+02
C@f[SRC]: +8.730e-06
C@f[DST]: -8.991e-06

ratio of max to min diagonal of laplacian : 2488.5800538625595
Solver build time: 0.002 seconds.
<b, LDLi b> is 7.337964990838643
PCG stopped due to small nr (norm(r)/nb) = 9.611228252258038e-13
nb = 1.414213562373095e6
PCG stopped after: 0.002 seconds and 18 iterations with relative error 9.611228252258038e-13.
rho = 8.921656665666626e-23
sum(C@f): -1.694e-21
J(f): +5.039e-08

Residual norm: 1.359e-06

[0.0 3.02319383400905e10 1.77243647289696e11 1.27741445454248e11; 3.02319383400905e10 0.0 0.0 0.0; 1.77243647289696e11 0.0 0.0 0.0; 1.27741445454248e11 0.0 0.0 0.0]
